BART text classification

In [1]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:0

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
import random
import string
import re
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
from transformers import get_scheduler
from tqdm.notebook import tqdm


from transformers import BartTokenizer, BartForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [5]:
from datasets import load_dataset
train_data,test_data = load_dataset("SetFit/tweet_sentiment_extraction",split='train[:50%]'),load_dataset("SetFit/tweet_sentiment_extraction",split='test[:50%]')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--tweet_sentiment_extraction-60a81958566f1313/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


In [6]:
train_data,test_data

(Dataset({
     features: ['textID', 'text', 'label', 'label_text'],
     num_rows: 13740
 }),
 Dataset({
     features: ['textID', 'text', 'label', 'label_text'],
     num_rows: 1767
 }))

In [7]:
train_df=pd.DataFrame(train_data)
test_df=pd.DataFrame(test_data)
train_df

,textID,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
...,...,...,...,...
13735,5b65d15956,http://twitpic.com/676hv - R.I.P Fristy just t...,0,negative
13736,f8afc0cc2e,one thing is good enough for me. friendship r...,2,positive
13737,1a41c7fa70,yep trying to beat the cold winter. Morning s...,2,positive
13738,d388f4b3e6,"Okay, now I`m listening to Jonathan Coulton an...",1,neutral


In [8]:
train_data=train_df[['text','label']]
train_data
test_data=test_df[['text','label']]
test_data

,text,label
0,Last session of the day http://twitpic.com/67ezh,1
1,Shanghai is also really exciting (precisely -...,2
2,"Recession hit Veronique Branquinho, she has to...",0
3,happy bday!,2
4,http://twitpic.com/4w75p - I like it!!,2
...,...,...
1762,They open at 9 a.m. And I hope to be there ...,1
1763,Is at work.... boo! The Pin-Ups will be play...,1
1764,has just got out the shower,1
1765,he`s back! meeting him in 40 mins,1


Cleaning the data

In [9]:
wl = WordNetLemmatizer()
ps = PorterStemmer()
nltk.download('stopwords')
sp = set(stopwords.words('english'))
def clean_text(texts):
    final_tmp=[]
    for text in texts:
        #lower and remove punctuation
        text=text.translate(str.maketrans('','',string.punctuation)).lower()
        
        #Remove Hyperlinks
        text=re.sub(r'http\S+','',text)
        
        #Remove numbers and words containing numbers
        text=' '.join([i for i in text.split() if i.isalpha()])
        
        #Now stop words
        text=' '.join(i for i in text.split() if i not in sp)
        
        final_tmp.append(text)
    return final_tmp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
train_data['text'] = clean_text(train_data['text'])
test_data['text'] = clean_text(test_data['text'])

<ipython-input-10-84dd5f1d107e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['text'] = clean_text(train_data['text'])
<ipython-input-10-84dd5f1d107e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['text'] = clean_text(test_data['text'])


In [11]:
train_data.head()

,text,label
0,id responded going,1
1,sooo sad miss san diego,0
2,boss bullying,0
3,interview leave alone,0
4,sons couldnt put releases already bought,0


In [12]:
test_data.head()

,text,label
0,last session day,1
1,shanghai also really exciting precisely skyscr...,2
2,recession hit veronique branquinho quit compan...,0
3,happy bday,2
4,like,2


## BART

In [13]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [14]:
class tweet_review(Dataset):
    def __init__(self, texts, label):
        self.texts = texts
        self.label = label

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        lab = self.label[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(lab, dtype=torch.long)
        }

In [15]:
train_dataset = tweet_review(train_data['text'], train_data['label'])
test_dataset = tweet_review(test_data['text'], test_data['label'])

Create DAta Loaders

In [17]:
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Model Initialization

In [19]:
# create model
model = BartForSequenceClassification.from_pretrained('facebook/bart-base',
                                                      output_attentions=False,
                                                      output_hidden_states=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps

Optimizer and learning rate

In [20]:
optimizer = AdamW(model.parameters(), lr=1e-5)
total_steps = len(train_loader) * 5  # 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=total_steps, gamma=0.9)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
device

device(type='cuda')

Training

In [22]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

Epoch 1/5 - Avg Loss: 0.7475
Epoch 2/5 - Avg Loss: 0.6216
Epoch 3/5 - Avg Loss: 0.5598
Epoch 4/5 - Avg Loss: 0.5080
Epoch 5/5 - Avg Loss: 0.4574


## testing

In [24]:
model.eval()
predictions = []
val = []
with torch.no_grad():
  for batch in test_loader:
    input = batch["input_ids"].to(device)
    att_mask = batch["attention_mask"].to(device)
    labels = batch["label"].to(device)
    outputs = model(input, attention_mask=att_mask)

    logits = outputs.logits
    _, pred = torch.max(logits, dim=1)

    predictions.extend(pred.detach().cpu().numpy())
    val.extend(labels.detach().cpu().numpy())


acc = accuracy_score(val, predictions)
classification_rep = classification_report(val, predictions)
print(f"Accuracy: {acc:.4f}")
print(classification_rep)

Accuracy: 0.7470
              precision    recall  f1-score   support

           0       0.69      0.81      0.75       497
           1       0.76      0.66      0.71       722
           2       0.79      0.81      0.80       548

    accuracy                           0.75      1767
   macro avg       0.75      0.76      0.75      1767
weighted avg       0.75      0.75      0.75      1767

